In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import date
from pandas_profiling import ProfileReport
from sklearn.preprocessing import StandardScaler

In [2]:
pd.set_option('display.max_columns', 500)

In [3]:
def category_crosstab(category, target, dataframe):
    df = pd.crosstab(dataframe[target], dataframe[category], normalize='columns').transpose().sort_values(by=0, ascending=False)
    
    return df

def kmeans_clusterer(category, target, dataframe, validation_dataframe, k_clusters = 0, colors = 'rainbow'):
    import matplotlib.pyplot as plt
    from kneed import KneeLocator
    from sklearn.cluster import KMeans
    from sklearn.metrics import silhouette_score
    from sklearn.preprocessing import StandardScaler

    
    columns= [category, target]
    cluster_data = dataframe[columns]
    
    conditional_probability = category_crosstab(category, target, cluster_data)
    cluster_data['conditional_probability'] = cluster_data[category].apply(lambda x: conditional_probability.loc[x][1])
    
    cluster_data['Income_count']=cluster_data['Income'].copy()
    
    clustering_data = cluster_data.groupby(category).agg({'Income':'sum',
                                                          'Income_count':'count',
                                                          'conditional_probability':'first'}).sort_values(by='Income_count')
    clustering_data
    
    
    scaler = StandardScaler()
    scaled_features = scaler.fit_transform(clustering_data)

    kmeans_kwargs = {
    "init": "random",
    "n_init": 10,
    "max_iter": 1500,
    "random_state": 42,
        }

    # A list holds the SSE values for each k
    sse = []
    
    number_unique_categories = len(dataframe[category].unique())
    if number_unique_categories > 10: 
        max_number_clusters = 11
    else:
        max_number_clusters = number_unique_categories

    for k in range(1, max_number_clusters):
        kmeans = KMeans(n_clusters=k, **kmeans_kwargs)
        kmeans.fit(scaled_features)
        sse.append(kmeans.inertia_)
    
  
    # Define Elbow point

    kl = KneeLocator(range(1, max_number_clusters), 
                     sse, 
                     curve="convex", 
                     direction="decreasing")
    
    if k_clusters == 0 : 
        number_cluster = kl.elbow
    else: 
        number_cluster = k_clusters
    
    # Run k means 
    
    kmeans= KMeans(n_clusters=number_cluster, **kmeans_kwargs)
    kmeans.fit(scaled_features)
    
    # assign kmeans labels to each category
    clustering_data['kmeans_cluster'] = kmeans.labels_
    
    
    # plot SSE (inertia) vs number of clusters - Improve
    plt.style.use('dark_background')
    plt.style.use('dark_background')
    fig, axes = plt.subplots(3,1,figsize=(10,20))
    fig.suptitle(category, fontsize=16)
    
        # first plot - inertia vs number of clusters
        
    x = range(1, max_number_clusters)
    y = sse
    #plt.style.use("fivethirtyeight")
    axes[0].plot(x, sse, marker = "D" )
    plt.sca(axes[0])
    plt.set_cmap(colors)
    plt.title('Inertia vs Number of Clusters')
    plt.xticks(range(1, max_number_clusters))
    plt.xlabel("Number of Clusters")
    plt.ylabel("SSE")
    plt.plot(x[number_cluster -1], y[number_cluster -1], 'ro')
    
        # second plot - number of individuals in class vs conditional probability
    
    sns.scatterplot(x='Income_count', 
                    y='conditional_probability', 
                    hue = 'kmeans_cluster' , 
                    data = clustering_data,
                    palette = colors,
                    ax = axes[1])
    
    plt.sca(axes[1])
    plt.title('Number of individuals in Category vs Conditional Probability')
    #plt.xticks(range(0, 100))
    # Set x-axis label
    plt.xlabel('Number of individuals in Category')
    # Set y-axis label
    plt.ylabel('Conditional Probability')
    
        # third plot - number of individuals in class that hit target vs conditional probability

    sns.scatterplot(x='Income', 
                    y='conditional_probability', 
                    hue = 'kmeans_cluster' , 
                    data = clustering_data,
                    palette = colors,
                    ax = axes[2])
    
    plt.sca(axes[2])
    plt.title('Number of individuals in Category w/ Target vs Conditional Probability')
    #plt.xticks(range(0, 100))
    # Set x-axis label
    plt.xlabel('Number of individuals in Category')
    # Set y-axis label
    plt.ylabel('Conditional Probability')
    
    #3d plotting 
    
    fig = plt.figure()
    fig.suptitle(category, fontsize=16)
    ax = plt.axes(projection='3d')
    
    # Data for three-dimensional scattered points
    zdata = clustering_data['conditional_probability']
    xdata = clustering_data['Income']
    ydata = clustering_data['Income_count']
    ax.scatter3D(xdata, ydata, zdata, c=clustering_data['kmeans_cluster'])#, cmap=colors);
    ax.set_xlabel('People in Category')
    ax.set_ylabel('People in Category w/target')
    ax.set_zlabel('Conditional Probability');
    
    
    display(clustering_data.sort_values(by='kmeans_cluster'))
    # Add cluster as dimension 
    new_category= category +' - Clustered'
    dataframe[new_category] = dataframe[category].apply(lambda x: clustering_data['kmeans_cluster'][x])
    validation_dataframe[new_category] = validation_dataframe[category].apply(lambda x: clustering_data['kmeans_cluster'][x])
    

    print('Process done')
    return kmeans

In [4]:
def simple_kmeans(clustering_data, max_k =10, k_clusters = 'else', visualize = True):
    import matplotlib.pyplot as plt
    from kneed import KneeLocator
    from sklearn.cluster import KMeans
    from sklearn.metrics import silhouette_score
    from sklearn.preprocessing import StandardScaler

    
    scaler = StandardScaler()
    scaled_features = scaler.fit_transform(clustering_data)
    scaled_df = pd.DataFrame(scaled_features)
    scaled_df.columns = clustering_data.columns

    kmeans_kwargs = {
    "init": "random",
    "n_init": 10,
    "max_iter": 1500,
    "random_state": 42,
        }

    # A list holds the SSE values for each k
    sse = []
    
    # Measuring SSE for different k cluster levels
    max_number_clusters = max_k
    
    
    for k in range(1, max_number_clusters):
        kmeans = KMeans(n_clusters=k, **kmeans_kwargs)
        kmeans.fit(scaled_features)
        sse.append(kmeans.inertia_)
        
        # Define Elbow point Automatically or manually. 

    kl = KneeLocator(range(1, max_number_clusters), 
                     sse, 
                     curve="convex", 
                     direction="decreasing")
    
    
    if k_clusters == 'else' : 
        number_cluster = kl.elbow
    else: 
        number_cluster = k_clusters
    
    if visualize == True:
    
        # plot SSE (inertia) vs number of clusters - Improve

        plt.style.use('fivethirtyeight') # set dark style, 'cause its simply better. 
        #plt.set_cmap('Set1')

        fig, axes = plt.subplots(1,1,figsize=(10,20)) #3 subplots, each with its row
        #fig.suptitle(category, fontsize=16)

            # first plot - inertia vs number of clusters

        x = range(1, max_number_clusters)
        y = sse
        #plt.style.use("fivethirtyeight")
        axes.plot(x, sse )

        plt.sca(axes) # select ax0 
        #plt.set_cmap(colors)

        plt.title('Inertia vs Number of Clusters') #title
        plt.xticks(range(1, max_number_clusters)) # xticks
        plt.xlabel("Number of Clusters") # xlabels
        plt.ylabel("SSE") # ylabels
        plt.plot(x[number_cluster -1], y[number_cluster -1], color='green', marker='X', 
                 linestyle='dashed', linewidth=15, markersize=25) # show that represents 

    
    
    # Run k means with right number of clusters 
    
    kmeans= KMeans(n_clusters=number_cluster, **kmeans_kwargs)
    kmeans.fit(scaled_features)
    
    # assign kmeans labels to each category
    clustering_data['kmeans_cluster'] = kmeans.labels_
    scaled_df['kmeans_cluster'] = kmeans.labels_
    
    display(clustering_data.sort_values(by='kmeans_cluster'))
    
    return scaled_df

def boxplot_cluster_compparisson(dataframe):

    variables = dataframe.iloc[:,:-1].columns
    number_variables = len(variables)
    height = number_variables*20

    fig, ax = plt.subplots(number_variables,1, figsize =(height,height))

    for variable, x in zip(variables, range(number_variables)): 
        sns.boxplot(x= dataframe.iloc[:,-1] ,
                    y = dataframe[variable],
                       ax = ax[x])     
    return fig

def kmeans_analysis(clustering_data, max_k =10, k_clusters = 'else', visualize = True):
    
    sc_df = simple_kmeans(clustering_data, max_k =10, k_clusters = 'else', visualize = True)
    graphs = boxplot_cluster_compparisson(sc_df)
    return graphs, sc_df

In [5]:
def plot_cumulative_significance_PCA(pca, plotTitle):
    """Takes the PCA model after fit and transform, plotting the cumulative significance of each component"""

    # figure and axes
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

    # draw plots
    ax1.plot(pca.explained_variance_, marker=".", markersize=12)
    ax2.plot(pca.explained_variance_ratio_, marker=".", markersize=12, label="Proportion")
    ax2.plot(np.cumsum(pca.explained_variance_ratio_), marker=".", markersize=12, linestyle="--", label="Cumulative")

    # customizations
    ax2.legend()
    ax1.set_title(plotTitle, fontsize=14)
    ax2.set_title("Variance Explained", fontsize=14)
    ax1.set_ylabel("Eigenvalue")
    ax2.set_ylabel("Proportion")
    ax1.set_xlabel("Components")
    ax2.set_xlabel("Components")
    ax1.set_xticks(range(0, pca.n_components_, 2))
    ax1.set_xticklabels(range(1, pca.n_components_ + 1, 2))
    ax2.set_xticks(range(0, pca.n_components_, 2))
    ax2.set_xticklabels(range(1, pca.n_components_ + 1, 2))

    plt.show()

In [6]:
def pca_analysis(subgroups_pca_dic, path_to_excel):
   # writer = pd.ExcelWriter('/Files/test_pca.xlsx')

    for subGroup in hood_subgroups_pca_dic.keys():
        columns = hood_subgroups_dic[subGroup]
        k = hood_subgroups_pca_dic[subGroup]

        #PCA fit
        pca = PCA(n_components=k)
        pca_feat = pca.fit_transform(pca_data[columns])

        #Creating dataframe
        pca_feat_names = [f"PC{i}" for i in range(k)]
        pca_df = pd.DataFrame(pca_feat, index=pca_data[columns].index, columns=pca_feat_names)

        # Reassigning df to contain pca variables
        pca_df = pd.concat([pca_data[columns], pca_df], axis=1)

        # Interpreting each Principal Component
        loadings = pca_df[columns + pca_feat_names].corr().loc[columns, pca_feat_names]
        print("\n\n\n//////////////////////////////////%s" % subGroup)
        display(loadings.style.applymap(_color_red_or_green))

        # Returning an excel file (sorry) with the analysis

        test='home_structures'

        loadings.style.applymap(_color_red_or_green).to_excel(writer, subGroup)
        #writer.save()

# 0. Loading our Data

In [7]:
df = pd.read_csv('Data/donors.csv')

C:\Users\josem\.conda\envs\python-standard\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (9,412) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [8]:
df.head()

,Unnamed: 0,ODATEDW,OSOURCE,TCODE,STATE,ZIP,MAILCODE,PVASTATE,DOB,NOEXCH,RECINHSE,RECP3,RECPGVG,RECSWEEP,MDMAUD,DOMAIN,HOMEOWNR,CHILD03,CHILD07,CHILD12,CHILD18,NUMCHLD,INCOME,GENDER,WEALTH1,HIT,MBCRAFT,MBGARDEN,MBBOOKS,MBCOLECT,MAGFAML,MAGFEM,MAGMALE,PUBGARDN,PUBCULIN,PUBHLTH,PUBDOITY,PUBNEWFN,PUBPHOTO,PUBOPP,DATASRCE,MALEMILI,MALEVET,VIETVETS,WWIIVETS,LOCALGOV,STATEGOV,FEDGOV,SOLP3,SOLIH,MAJOR,WEALTH2,GEOCODE,COLLECT1,VETERANS,BIBLE,CATLG,HOMEE,PETS,CDPLAY,STEREO,PCOWNERS,PHOTO,CRAFTS,FISHER,GARDENIN,BOATS,WALKER,KIDSTUFF,CARDS,PLATES,LIFESRC,PEPSTRFL,POP901,POP902,POP903,POP90C1,POP90C2,POP90C3,POP90C4,POP90C5,ETH1,ETH2,ETH3,ETH4,ETH5,ETH6,ETH7,ETH8,ETH9,ETH10,ETH11,ETH12,ETH13,ETH14,ETH15,ETH16,AGE901,AGE902,AGE903,AGE904,AGE905,AGE906,AGE907,CHIL1,CHIL2,CHIL3,AGEC1,AGEC2,AGEC3,AGEC4,AGEC5,AGEC6,AGEC7,CHILC1,CHILC2,CHILC3,CHILC4,CHILC5,HHAGE1,HHAGE2,HHAGE3,HHN1,HHN2,HHN3,HHN4,HHN5,HHN6,MARR1,MARR2,MARR3,MARR4,HHP1,HHP2,DW1,DW2,DW3,DW4,DW5,DW6,DW7,DW8,DW9,HV1,HV2,HV3,HV4,HU1,HU2,HU3,HU4,HU5,HHD1,HHD2,HHD3,HHD4,HHD5,HHD6,HHD7,HHD8,HHD9,HHD10,HHD11,HHD12,ETHC1,ETHC2,ETHC3,ETHC4,ETHC5,ETHC6,HVP1,HVP2,HVP3,HVP4,HVP5,HVP6,HUR1,HUR2,RHP1,RHP2,RHP3,RHP4,HUPA1,HUPA2,HUPA3,HUPA4,HUPA5,HUPA6,HUPA7,RP1,RP2,RP3,RP4,MSA,ADI,DMA,IC1,IC2,IC3,IC4,IC5,IC6,IC7,IC8,IC9,IC10,IC11,IC12,IC13,IC14,IC15,IC16,IC17,IC18,IC19,IC20,IC21,IC22,IC23,HHAS1,HHAS2,HHAS3,HHAS4,MC1,MC2,MC3,TPE1,TPE2,TPE3,TPE4,TPE5,TPE6,TPE7,TPE8,TPE9,PEC1,PEC2,TPE10,TPE11,TPE12,TPE13,LFC1,LFC2,LFC3,LFC4,LFC5,LFC6,LFC7,LFC8,LFC9,LFC10,OCC1,OCC2,OCC3,OCC4,OCC5,OCC6,OCC7,OCC8,OCC9,OCC10,OCC11,OCC12,OCC13,EIC1,EIC2,EIC3,EIC4,EIC5,EIC6,EIC7,EIC8,EIC9,EIC10,EIC11,EIC12,EIC13,EIC14,EIC15,EIC16,OEDC1,OEDC2,OEDC3,OEDC4,OEDC5,OEDC6,OEDC7,EC1,EC2,EC3,EC4,EC5,EC6,EC7,EC8,SEC1,SEC2,SEC3,SEC4,SEC5,AFC1,AFC2,AFC3,AFC4,AFC5,AFC6,VC1,VC2,VC3,VC4,ANC1,ANC2,ANC3,ANC4,ANC5,ANC6,ANC7,ANC8,ANC9,ANC10,ANC11,ANC12,ANC13,ANC14,ANC15,POBC1,POBC2,LSC1,LSC2,LSC3,LSC4,VOC1,VOC2,VOC3,HC1,HC2,HC3,HC4,HC5,HC6,HC7,HC8,HC9,HC10,HC11,HC12,HC13,HC14,HC15,HC16,HC17,HC18,HC19,HC20,HC21,MHUC1,MHUC2,AC1,AC2,ADATE_2,ADATE_3,ADATE_4,ADATE_5,ADATE_6,ADATE_7,ADATE_8,ADATE_9,ADATE_10,ADATE_11,ADATE_12,ADATE_13,ADATE_14,ADATE_15,ADATE_16,ADATE_17,ADATE_18,ADATE_19,ADATE_20,ADATE_21,ADATE_22,ADATE_23,ADATE_24,RFA_2,RFA_3,RFA_4,RFA_5,RFA_6,RFA_7,RFA_8,RFA_9,RFA_10,RFA_11,RFA_12,RFA_13,RFA_14,RFA_15,RFA_16,RFA_17,RFA_18,RFA_19,RFA_20,RFA_21,RFA_22,RFA_23,RFA_24,CARDPROM,MAXADATE,NUMPROM,CARDPM12,NUMPRM12,RDATE_3,RDATE_4,RDATE_5,RDATE_6,RDATE_7,RDATE_8,RDATE_9,RDATE_10,RDATE_11,RDATE_12,RDATE_13,RDATE_14,RDATE_15,RDATE_16,RDATE_17,RDATE_18,RDATE_19,RDATE_20,RDATE_21,RDATE_22,RDATE_23,RDATE_24,RAMNT_3,RAMNT_4,RAMNT_5,RAMNT_6,RAMNT_7,RAMNT_8,RAMNT_9,RAMNT_10,RAMNT_11,RAMNT_12,RAMNT_13,RAMNT_14,RAMNT_15,RAMNT_16,RAMNT_17,RAMNT_18,RAMNT_19,RAMNT_20,RAMNT_21,RAMNT_22,RAMNT_23,RAMNT_24,RAMNTALL,NGIFTALL,CARDGIFT,MINRAMNT,MINRDATE,MAXRAMNT,MAXRDATE,LASTGIFT,LASTDATE,FISTDATE,NEXTDATE,TIMELAG,AVGGIFT,CONTROLN,HPHONE_D,RFA_2R,RFA_2F,RFA_2A,MDMAUD_R,MDMAUD_F,MDMAUD_A,GEOCODE2
0,0,2009-01-01,GRI,0,IL,61081,,,1957-12-01,0,,,,,XXXX,T2,,,,,,NaN,NaN,F,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,0,39,34,18,10,2,1,,,,5.0,,,,,,,,,,,,,,,,,,,,,X,992,264,332,0,35,65,47,53,92,1,0,0,11,0,0,0,0,0,0,0,11,0,0,0,39,48,51,40,50,54,25,31,42,27,11,14,18,17,13,11,15,12,11,34,25,18,26,10,23,18,33,49,28,12,4,61,7,12,19,198,276,97,95,2,2,0,0,7,7,0,479,635,3,2,86,14,96,4,7,38,80,70,32,84,16,6,2,5,9,15,3,17,50,25,0,0,0,2,7,13,27,47,0,1,61,58,61,15,4,2,0,0,14,1,0,0,2,5,17,73,0.0,177.0,682.0,307,318,349,378,12883,13,23,23,23,15,1,0,0,1,4,25,24,26,17,2,0,0,2,28,4,51,1,46,54,3,88,8,0,0,0,0,0,0,4,1,13,14,16,2,45,56,64,50,64,44,62,53,99,0,0,9,3,8,13,9,0,3,9,3,15,19,5,4,3,0,3,41,1,0,7,13,6,5,0,4,9,4,1,3,10,2,1,7,78,2,0,120,16,10,39,21,8,4,3,5,20,3,19,4,0,0,0,18,39,0,34,23,18,16,1,4,0,23,0,0,5,1,0,0,0,0,0,2,0,3,74,88,8,0,4,96,77,19,13,31,5,14,14,31,54,46,0,0,90,0,10,0,0,0,33,65,40,99,99,6,2,10,7,2017-06-01,2016-06-01,2016-04-01,2016-04-01,2016-03-01,

In [9]:
df['ADATE_2'].value_counts()

2017-06-01    95399
2017-04-01       13
Name: ADATE_2, dtype: int64

In [10]:
df['RFA_2'].value_counts() # All lapsed...?

L1F    30380
L1G    12384
L2F    10961
L3E     7767
L2E     4989
L4D     4914
L1E     4911
L2G     4595
L4E     3972
L3F     3523
L3D     2498
L4F     2100
L3G     1503
L4G      915
Name: RFA_2, dtype: int64

In [11]:
df['ADATE_3'].value_counts()

2016-06-01    93444
2016-04-01       18
Name: ADATE_3, dtype: int64

In [12]:
df['RFA_3'].value_counts()

A1F    21950
A1G     9184
A2F     6283
F1F     5973
A1E     5119
       ...  
S2B        2
S3C        2
A2C        1
N1C        1
S3B        1
Name: RFA_3, Length: 71, dtype: int64

In [13]:
rfa_variables = [
#'RFA_2',
'RFA_3',
'RFA_4',
'RFA_5',
'RFA_6',
'RFA_7',
'RFA_8',
'RFA_9',
'RFA_10',
'RFA_11',
'RFA_12',
'RFA_13',
'RFA_14',
'RFA_15',
'RFA_16',
'RFA_17',
'RFA_18',
'RFA_19',
'RFA_20',
'RFA_21',
'RFA_22',
'RFA_23',
'RFA_24',        
]

receiving_dates = [
'RDATE_3',
'RDATE_4',
'RDATE_5',
'RDATE_6',
'RDATE_7',
'RDATE_8',
'RDATE_9',
'RDATE_10',
'RDATE_11',
'RDATE_12',
'RDATE_13',
'RDATE_14',
'RDATE_15',
'RDATE_16',
'RDATE_17',
'RDATE_18',
'RDATE_19',
'RDATE_20',
'RDATE_21',
'RDATE_22',
'RDATE_23',
'RDATE_24',
]

In [14]:
df[rfa_variables]['RFA_10'].unique()
df.drop(columns = ['RFA_2'], inplace=True)

In [15]:
df

,Unnamed: 0,ODATEDW,OSOURCE,TCODE,STATE,ZIP,MAILCODE,PVASTATE,DOB,NOEXCH,RECINHSE,RECP3,RECPGVG,RECSWEEP,MDMAUD,DOMAIN,HOMEOWNR,CHILD03,CHILD07,CHILD12,CHILD18,NUMCHLD,INCOME,GENDER,WEALTH1,HIT,MBCRAFT,MBGARDEN,MBBOOKS,MBCOLECT,MAGFAML,MAGFEM,MAGMALE,PUBGARDN,PUBCULIN,PUBHLTH,PUBDOITY,PUBNEWFN,PUBPHOTO,PUBOPP,DATASRCE,MALEMILI,MALEVET,VIETVETS,WWIIVETS,LOCALGOV,STATEGOV,FEDGOV,SOLP3,SOLIH,MAJOR,WEALTH2,GEOCODE,COLLECT1,VETERANS,BIBLE,CATLG,HOMEE,PETS,CDPLAY,STEREO,PCOWNERS,PHOTO,CRAFTS,FISHER,GARDENIN,BOATS,WALKER,KIDSTUFF,CARDS,PLATES,LIFESRC,PEPSTRFL,POP901,POP902,POP903,POP90C1,POP90C2,POP90C3,POP90C4,POP90C5,ETH1,ETH2,ETH3,ETH4,ETH5,ETH6,ETH7,ETH8,ETH9,ETH10,ETH11,ETH12,ETH13,ETH14,ETH15,ETH16,AGE901,AGE902,AGE903,AGE904,AGE905,AGE906,AGE907,CHIL1,CHIL2,CHIL3,AGEC1,AGEC2,AGEC3,AGEC4,AGEC5,AGEC6,AGEC7,CHILC1,CHILC2,CHILC3,CHILC4,CHILC5,HHAGE1,HHAGE2,HHAGE3,HHN1,HHN2,HHN3,HHN4,HHN5,HHN6,MARR1,MARR2,MARR3,MARR4,HHP1,HHP2,DW1,DW2,DW3,DW4,DW5,DW6,DW7,DW8,DW9,HV1,HV2,HV3,HV4,HU1,HU2,HU3,HU4,HU5,HHD1,HHD2,HHD3,HHD4,HHD5,HHD6,HHD7,HHD8,HHD9,HHD10,HHD11,HHD12,ETHC1,ETHC2,ETHC3,ETHC4,ETHC5,ETHC6,HVP1,HVP2,HVP3,HVP4,HVP5,HVP6,HUR1,HUR2,RHP1,RHP2,RHP3,RHP4,HUPA1,HUPA2,HUPA3,HUPA4,HUPA5,HUPA6,HUPA7,RP1,RP2,RP3,RP4,MSA,ADI,DMA,IC1,IC2,IC3,IC4,IC5,IC6,IC7,IC8,IC9,IC10,IC11,IC12,IC13,IC14,IC15,IC16,IC17,IC18,IC19,IC20,IC21,IC22,IC23,HHAS1,HHAS2,HHAS3,HHAS4,MC1,MC2,MC3,TPE1,TPE2,TPE3,TPE4,TPE5,TPE6,TPE7,TPE8,TPE9,PEC1,PEC2,TPE10,TPE11,TPE12,TPE13,LFC1,LFC2,LFC3,LFC4,LFC5,LFC6,LFC7,LFC8,LFC9,LFC10,OCC1,OCC2,OCC3,OCC4,OCC5,OCC6,OCC7,OCC8,OCC9,OCC10,OCC11,OCC12,OCC13,EIC1,EIC2,EIC3,EIC4,EIC5,EIC6,EIC7,EIC8,EIC9,EIC10,EIC11,EIC12,EIC13,EIC14,EIC15,EIC16,OEDC1,OEDC2,OEDC3,OEDC4,OEDC5,OEDC6,OEDC7,EC1,EC2,EC3,EC4,EC5,EC6,EC7,EC8,SEC1,SEC2,SEC3,SEC4,SEC5,AFC1,AFC2,AFC3,AFC4,AFC5,AFC6,VC1,VC2,VC3,VC4,ANC1,ANC2,ANC3,ANC4,ANC5,ANC6,ANC7,ANC8,ANC9,ANC10,ANC11,ANC12,ANC13,ANC14,ANC15,POBC1,POBC2,LSC1,LSC2,LSC3,LSC4,VOC1,VOC2,VOC3,HC1,HC2,HC3,HC4,HC5,HC6,HC7,HC8,HC9,HC10,HC11,HC12,HC13,HC14,HC15,HC16,HC17,HC18,HC19,HC20,HC21,MHUC1,MHUC2,AC1,AC2,ADATE_2,ADATE_3,ADATE_4,ADATE_5,ADATE_6,ADATE_7,ADATE_8,ADATE_9,ADATE_10,ADATE_11,ADATE_12,ADATE_13,ADATE_14,ADATE_15,ADATE_16,ADATE_17,ADATE_18,ADATE_19,ADATE_20,ADATE_21,ADATE_22,ADATE_23,ADATE_24,RFA_3,RFA_4,RFA_5,RFA_6,RFA_7,RFA_8,RFA_9,RFA_10,RFA_11,RFA_12,RFA_13,RFA_14,RFA_15,RFA_16,RFA_17,RFA_18,RFA_19,RFA_20,RFA_21,RFA_22,RFA_23,RFA_24,CARDPROM,MAXADATE,NUMPROM,CARDPM12,NUMPRM12,RDATE_3,RDATE_4,RDATE_5,RDATE_6,RDATE_7,RDATE_8,RDATE_9,RDATE_10,RDATE_11,RDATE_12,RDATE_13,RDATE_14,RDATE_15,RDATE_16,RDATE_17,RDATE_18,RDATE_19,RDATE_20,RDATE_21,RDATE_22,RDATE_23,RDATE_24,RAMNT_3,RAMNT_4,RAMNT_5,RAMNT_6,RAMNT_7,RAMNT_8,RAMNT_9,RAMNT_10,RAMNT_11,RAMNT_12,RAMNT_13,RAMNT_14,RAMNT_15,RAMNT_16,RAMNT_17,RAMNT_18,RAMNT_19,RAMNT_20,RAMNT_21,RAMNT_22,RAMNT_23,RAMNT_24,RAMNTALL,NGIFTALL,CARDGIFT,MINRAMNT,MINRDATE,MAXRAMNT,MAXRDATE,LASTGIFT,LASTDATE,FISTDATE,NEXTDATE,TIMELAG,AVGGIFT,CONTROLN,HPHONE_D,RFA_2R,RFA_2F,RFA_2A,MDMAUD_R,MDMAUD_F,MDMAUD_A,GEOCODE2
0,0,2009-01-01,GRI,0,IL,61081,,,1957-12-01,0,,,,,XXXX,T2,,,,,,NaN,NaN,F,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,0,39,34,18,10,2,1,,,,5.0,,,,,,,,,,,,,,,,,,,,,X,992,264,332,0,35,65,47,53,92,1,0,0,11,0,0,0,0,0,0,0,11,0,0,0,39,48,51,40,50,54,25,31,42,27,11,14,18,17,13,11,15,12,11,34,25,18,26,10,23,18,33,49,28,12,4,61,7,12,19,198,276,97,95,2,2,0,0,7,7,0,479,635,3,2,86,14,96,4,7,38,80,70,32,84,16,6,2,5,9,15,3,17,50,25,0,0,0,2,7,13,27,47,0,1,61,58,61,15,4,2,0,0,14,1,0,0,2,5,17,73,0.0,177.0,682.0,307,318,349,378,12883,13,23,23,23,15,1,0,0,1,4,25,24,26,17,2,0,0,2,28,4,51,1,46,54,3,88,8,0,0,0,0,0,0,4,1,13,14,16,2,45,56,64,50,64,44,62,53,99,0,0,9,3,8,13,9,0,3,9,3,15,19,5,4,3,0,3,41,1,0,7,13,6,5,0,4,9,4,1,3,10,2,1,7,78,2,0,120,16,10,39,21,8,4,3,5,20,3,19,4,0,0,0,18,39,0,34,23,18,16,1,4,0,23,0,0,5,1,0,0,0,0,0,2,0,3,74,88,8,0,4,96,77,19,13,31,5,14,14,31,54,46,0,0,90,0,10,0,0,0,33,65,40,99,99,6,2,10,7,2017-06-01,2016-06-01,2016-04-01,2016-04-01,2016-03-01,2016-0

In [16]:
recency_df = df[rfa_variables].copy()


for x in recency_df.columns: 
    recency_df[x] = recency_df[x].apply(lambda y: y[0])



In [28]:
recency_df

,RFA_3,RFA_4,RFA_5,RFA_6,RFA_7,RFA_8,RFA_9,RFA_10,RFA_11,RFA_12,RFA_13,RFA_14,RFA_15,RFA_16,RFA_17,RFA_18,RFA_19,RFA_20,RFA_21,RFA_22,RFA_23,RFA_24
0,S,S,S,S,S,S,S,S,S,S,S,S,S,S,S,S,S,S,S,S,S,S
1,A,A,A,A,A,A,A,A,A,A,,,,L,,,N,N,N,N,,F
2,S,S,S,S,S,S,S,,S,S,S,S,S,S,,S,S,,,S,S,S
3,S,S,S,S,S,S,S,,S,S,S,S,S,S,S,S,A,A,A,A,,
4,A,A,A,A,I,A,A,L,A,A,L,L,,L,A,A,A,A,A,I,A,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95407,F,F,,F,P,P,,,,,,,,,,,,,,,,
95408,,,,,P,P,,,,,,,,,,,,,,,,
95409,S,S,S,S,,A,N,N,N,N,N,N,N,F,,F,,,P,P,,
95410,S,A,S,S,S,S,S,S,S,S,S,S,S,S,S,S,A,A,A,A,S,S


In [ ]:
list(test_df['RFA_10'].unique())

In [20]:
counter = pd.DataFrame(index=['L', 'I', 'F', 'N', 'A', 'S'],
                columns=test_df.columns)

for column in counter.columns:
    for index in list(test_df[column].unique()): 
        try:
            counter.loc[index, column] = test_df[column].value_counts()[index]
        except KeyError:
            pass
counter


,RFA_3,RFA_4,RFA_5,RFA_6,RFA_7,RFA_8,RFA_9,RFA_10,RFA_11,RFA_12,RFA_13,RFA_14,RFA_15,RFA_16,RFA_17,RFA_18,RFA_19,RFA_20,RFA_21,RFA_22,RFA_23,RFA_24
L,478,440,95,5281,5269,5797,7499,274,7459,7446,91,6263,NaN,9334,7090,7533,8365,977,7133,7977,69,4059
I,NaN,NaN,NaN,1056,256,274,751,150,1174,1152,167,40,NaN,2535,1266,1513,846,11,629,1514,1,8
F,8316,8361,NaN,10037,7135,7325,4039,3974,4589,4711,2962,5680,NaN,5482,4359,4668,3476,2393,2773,3210,1082,6690
N,5955,6107,3272,8596,8360,8854,11292,8262,9874,10025,6265,6662,2437,6572,6254,6868,9888,5983,6719,8094,2431,3142
A,59567,59353,41455,48176,43406,45179,40978,35785,39904,40462,28561,38306,14900,33507,29125,31095,30416,24951,26546,29086,20101,29305
S,19146,18839,17000,18657,17206,19218,18706,14219,18145,18298,17139,16042,12597,17197,14422,16304,16347,10894,12023,15193,15429,14272
,1950,2312,33590,3557,8874,3511,11245,32748,10422,8923,40219,18867,65477,20417,27650,21263,24492,50200,35212,25648,56274,36973
P,NaN,NaN,NaN,4,4905,5216,887,NaN,3845,4142,NaN,3547,NaN,5,5220,5755,1548,NaN,4377,4611,NaN,938
U,NaN,NaN,NaN,48,1,38,15,NaN,NaN,253,8,5,1,363,26,413,34,3,NaN,79,3,25
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,NaN


In [26]:
counter.sum(axis=1)

L     98929.0
I     13343.0
F    101262.0
N    151912.0
A    790164.0
S    357293.0
     539824.0
P     45000.0
U      1315.0
2         6.0
1        11.0
3         4.0
4         1.0
dtype: float64

In [ ]:
counter.fillna(0).transpose().iloc[::-1].iloc[:-1,:5].plot()

In [19]:
test_df = df[rfa_variables].copy()

for x in test_df.columns: 
    test_df[x] = test_df[x].apply(lambda y: y[0])

test_df

,RFA_3,RFA_4,RFA_5,RFA_6,RFA_7,RFA_8,RFA_9,RFA_10,RFA_11,RFA_12,RFA_13,RFA_14,RFA_15,RFA_16,RFA_17,RFA_18,RFA_19,RFA_20,RFA_21,RFA_22,RFA_23,RFA_24
0,S,S,S,S,S,S,S,S,S,S,S,S,S,S,S,S,S,S,S,S,S,S
1,A,A,A,A,A,A,A,A,A,A,,,,L,,,N,N,N,N,,F
2,S,S,S,S,S,S,S,,S,S,S,S,S,S,,S,S,,,S,S,S
3,S,S,S,S,S,S,S,,S,S,S,S,S,S,S,S,A,A,A,A,,
4,A,A,A,A,I,A,A,L,A,A,L,L,,L,A,A,A,A,A,I,A,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95407,F,F,,F,P,P,,,,,,,,,,,,,,,,
95408,,,,,P,P,,,,,,,,,,,,,,,,
95409,S,S,S,S,,A,N,N,N,N,N,N,N,F,,F,,,P,P,,
95410,S,A,S,S,S,S,S,S,S,S,S,S,S,S,S,S,A,A,A,A,S,S


# 1.Analysing Data

## Donator classification

- RFA_2R    --> Recency code for RFA_2
- RFA_2F    --> Frequency code for RFA_2
- RFA_2A    --> Donation Amount code for RFA_2

In [ ]:
columns_donator_classification = [
    'RFA_2R',
    'RFA_2F',
    'RFA_2A'
]

donors_subgroups_dic = {'columns_donator_classification' : columns_donator_classification}


In [ ]:
rfa_2= df[columns_donator_classification]
rfa_2

In [ ]:
rfa_2.iloc[:,0].value_counts()

In [ ]:
rfa2_clusterable = pd.get_dummies(rfa_2)
rfa2_normalized_clustered = simple_kmeans(rfa2_clusterable)

In [ ]:
boxplot_cluster_compparisson(rfa2_normalized_clustered)

In [ ]:
from sklearn.decomposition import PCA

writer = pd.ExcelWriter('../Files/demographic_pca.xlsx')

columns = rfa2_clusterable.columns
pca = PCA()
pca.fit_transform(rfa2_normalized_clustered[columns].iloc[:,:-1])

# Plot PCA Groups
plot_cumulative_significance_PCA(pca, 'title')


In [ ]:
def _color_red_or_green(val):
    if val < -0.45:
        color = 'background-color: red'
    elif val > 0.45:
        color = 'background-color: green'
    else:
        color = ''
    return color


pca = PCA(n_components=5)

columns = rfa2_normalized_clustered.columns[:-1]

pca_data = rfa2_normalized_clustered.copy()
pca_feat = pca.fit_transform(pca_data[columns].iloc[:,:-1])

#Creating dataframe
pca_feat_names = [f"PC{i}" for i in range(5)]
pca_df = pd.DataFrame(pca_feat, index=pca_data[columns].index, columns=pca_feat_names)

# Reassigning df to contain pca variables
pca_df = pd.concat([pca_data[columns], pca_df], axis=1)

pca_df.head()


# Interpreting each Principal Component
loadings = pca_df.corr().loc[columns, pca_feat_names]
#print("\n\n\n//////////////////////////////////%s" % subGroup)
display(loadings.style.applymap(_color_red_or_green))
    
# Returning an excel file (sorry) with the analysis
    

#loadings.style.applymap(_color_red_or_green).to_excel(writer, subGroup)
#writer.save()


In [ ]:
df[columns_donator_classification].isna().sum()

In [ ]:
df[columns_donator_classification] = df[columns_donator_classification].astype('object')

 ## Donation history

- RAMNTALL  -->  Dollar amount of lifetime gifts to date
- NGIFTALL  -->  Number of lifetime gifts to date
- CARDGIFT  -->  Number of lifetime gifts to card promotions to date
- MINRAMNT  -->  Dollar amount of smallest gift to date
- MINRDATE  -->  Date associated with the smallest gift to date
- MAXRAMNT  -->  Dollar amount of largest gift to date
- MAXRDATE  -->  Date associated with the largest gift to date
- LASTGIFT  -->  Dollar amount of most recent gift
- LASTDATE  -->  Date associated with the most recent gift
- FISTDATE  -->  Date of first gift
- NEXTDATE  -->  Date of second gift
- TIMELAG  -->   Number of months between first and second gift
- AVGGIFT  -->   Average dollar amount of gifts to date

In [ ]:
columns_donation_history = [
    'RAMNTALL', # Dollar amount of lifetime gifts to date
    'NGIFTALL', # Number of lifetime gifts to date
    'CARDGIFT', # Dollar amount of smallest gift to date
    'MINRAMNT', # Date associated with the smallest gift to date
    'MINRDATE', # Date associated with the largest gift to date
    'MAXRAMNT', # Dollar amount of largest gift to date
    'MAXRDATE', # Dollar amount of most recent gift
    'LASTGIFT', # Date associated with the largest gift to date
    'LASTDATE', # Date associated with the most recent gift
    'FISTDATE', # Date of first gift
    'NEXTDATE', # Date of second gift 
    'TIMELAG', # Number of months between first and second gift 
    'AVGGIFT' # Average dollar amount of gifts to date 
]

In [ ]:
df[columns_donation_history].head()

In [ ]:
df[columns_donation_history].isna().sum()

In [ ]:
#drop rows with FISTDATE == NaN
df.dropna(subset=['FISTDATE'], inplace=True)
df[columns_donation_history].isna().sum()

In [ ]:
#fill time lag between first and second donation to 0
df['TIMELAG'].fillna(0, inplace=True)
df[columns_donation_history].isna().sum()

In [ ]:
# Make sure datatype is correct
df[columns_donation_history].dtypes

In [ ]:
#fix dtypes
df[columns_donation_history] =\
    df[columns_donation_history].astype(
        {
            'RAMNTALL': 'int64',
            'MINRAMNT': 'int64',
            'MINRDATE': 'datetime64',
            'MAXRAMNT': 'int64',
            'MAXRDATE': 'datetime64',
            'LASTGIFT': 'int64',
            'LASTDATE': 'datetime64',
            'FISTDATE': 'datetime64',
            'NEXTDATE': 'datetime64',
            'TIMELAG': 'int64',
            'AVGGIFT': 'int64'
        }
    )

In [ ]:
df[columns_donation_history].head()

In [ ]:
#generate time delta between biggest and smallest donation
timelag_days = list(map(lambda delta: delta.days, np.abs(df['MINRDATE'] - df['MAXRDATE'])))

new_column = 'TIMEDELTA_SMALLEST_BIGGEST_DONATION'
df[new_column] = timelag_days
columns_donation_history.append(new_column)

In [ ]:
#generate time delta between biggest and first donation
timelag_days = list(map(lambda delta: delta.days, np.abs(df['MAXRDATE'] - df['FISTDATE'])))

new_column = 'TIMEDELTA_FIRST_BIGGEST_DONATION'
df[new_column] = timelag_days
columns_donation_history.append(new_column)

In [ ]:
#generate time delta between first and last donation
timelag_days = list(map(lambda delta: delta.days, np.abs(df['LASTDATE'] - df['FISTDATE'])))

new_column = 'TIMEDELTA_FIRST_LAST_DONATION'
df[new_column] = timelag_days
columns_donation_history.append(new_column)

In [ ]:
columns_to_remove = ['MINRDATE','MAXRDATE','LASTDATE','FISTDATE','NEXTDATE']
df.drop(columns=columns_to_remove, inplace=True)

columns_donation_history = [col for col in columns_donation_history if col not in columns_to_remove]

In [ ]:
#Add "Average donation value per gift"'s column
new_column = 'AVG_GIFT_VAL'
columns_donation_history.append(new_column) 

df[new_column] = df['RAMNTALL'] / df['NGIFTALL']
df[new_column].head()

In [ ]:
df[columns_donation_history].head()

In [ ]:
df[columns_donation_history].columns

In [ ]:
kmeans_analysis(df[columns_donation_history], max_k=20) 

## Merging promotions by year
### Number gifts per year

In [ ]:
columns_promotions_2014 = ['RDATE_23','RDATE_24']

columns_promotions_2015 = ['RDATE_13','RDATE_14','RDATE_15','RDATE_16','RDATE_17','RDATE_18','RDATE_19','RDATE_20','RDATE_21','RDATE_22','RDATE_23','RDATE_24']

columns_promotions_2016 = ['RDATE_3','RDATE_4','RDATE_5','RDATE_6','RDATE_7','RDATE_8','RDATE_9','RDATE_10','RDATE_11','RDATE_12']

In [ ]:
#Getting number of donations per year
nr_donations_2014 = df[columns_promotions_2014].isna().sum(axis=1)
nr_donations_2015 = df[columns_promotions_2015].isna().sum(axis=1)
nr_donations_2016 = df[columns_promotions_2016].isna().sum(axis=1)
donations = pd.DataFrame([nr_donations_2014, nr_donations_2015, nr_donations_2016]).transpose()

columns_nr_donations_perYear = ['TOT_DON_14','TOT_DON_15','TOT_DON_16']

In [ ]:
df[columns_nr_donations_perYear] = donations
df[columns_nr_donations_perYear].head()

In [ ]:
donations


In [ ]:
kmeans_analysis(df[columns_nr_donations_perYear])

### Total gift amount per year

In [ ]:
columns_promotions_2014 = ['RAMNT_23','RAMNT_24']

columns_promotions_2015 = ['RAMNT_13','RAMNT_14','RAMNT_15','RAMNT_16','RAMNT_17','RAMNT_18','RAMNT_19','RAMNT_20','RAMNT_21','RAMNT_22','RAMNT_23','RAMNT_24']

columns_promotions_2016 = ['RAMNT_3','RAMNT_4','RAMNT_5','RAMNT_6','RAMNT_7','RAMNT_8','RAMNT_9','RAMNT_10','RAMNT_11','RAMNT_12']

In [ ]:
#Getting number of donations per year
nr_donations_2014 = df[columns_promotions_2014].sum(axis=1)
nr_donations_2015 = df[columns_promotions_2015].sum(axis=1)
nr_donations_2016 = df[columns_promotions_2016].sum(axis=1)
donations = pd.DataFrame([nr_donations_2014, nr_donations_2015, nr_donations_2016]).transpose()

columns_total_donation_amount_perYear = ['TOT_DON_AMNT_14','TOT_DON_AMNT_15','TOT_DON_AMNT_16']

In [ ]:
df[columns_total_donation_amount_perYear] = donations
df[columns_total_donation_amount_perYear].head()

In [ ]:
kmeans_analysis(df[columns_total_donation_amount_perYear])

## Adding return rate per donator
- **CARDPROM**  -->   Lifetime number of card promotions received to
                    date. Card promotions are promotion type FS, GK,
                    TK, SK, NK, XK, UF, UU.

- **MAXADATE**  -->   Date of the most recent promotion received (in
                    YYMM, Year/Month format)

- **NUMPROM**   -->   Lifetime number of promotions received to date

- **CARDPM12**  -->   Number of card promotions received in the last
                    12 months (in terms of calendar months translates
                    into 1603-1702)
                    
- **NUMPRM12**  -->   Number of promotions received in the last 12
                    months (in terms of calendar months translates
                    into 1603-1702)

In [ ]:
columns_overall_promotion_targeting = [
    'CARDPROM',
    'MAXADATE',
    'NUMPROM',
    'CARDPM12',
    'NUMPRM12',
]

columns_return_rates = []

In [ ]:
df[columns_overall_promotion_targeting].head()

In [ ]:
#Adding conversion rate into df
new_column = 'CONVERSION_RATE'
columns_return_rates.append(new_column)

df[new_column] = df['NGIFTALL'] / df['NUMPROM']
df[new_column].head()

In [ ]:
#Adding conversion rate into df
new_column = 'GIFT_AVG_AMNT'
columns_return_rates.append(new_column)

df[new_column] = df['NGIFTALL'] / df['NUMPROM']
df[new_column].head()

In [ ]:
#Adding average gift per promotion targeting
new_column = 'GIFT_AVG_AMNT_PER_PROMOTION'
columns_return_rates.append(new_column)

df[new_column] = df['RAMNTALL'] / df['NUMPROM']
df[new_column].head()

## Small Recap

In [ ]:
modified_columns =\
    columns_donator_classification +\
    columns_donation_history +\
    columns_nr_donations_perYear +\
    columns_total_donation_amount_perYear +\
    columns_return_rates

df[modified_columns].head()

In [ ]:
pd.get_dummies(df[modified_columns]).columns

In [ ]:
corr = df[modified_columns].corr()

plt.figure(figsize=(20,20))
sns.heatmap(corr, annot=True, linewidths=1)
plt.show()